In [157]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'


In [158]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [159]:
dataset = pd.read_csv("heart.csv")
dataset.head()


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [160]:
dataset.dtypes


age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object

In [161]:
X = dataset.iloc[:, :-1]
X.head()


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2


In [162]:
y = dataset.iloc[:, [-1]]
y.head()


,target
0,1
1,1
2,1
3,1
4,1


In [163]:
X_ohe = pd.get_dummies(X, columns=["cp", "restecg", "slope", "thal"])


In [164]:
X_ohe.columns


Index(['age', 'sex', 'trestbps', 'chol', 'fbs', 'thalach', 'exang', 'oldpeak',
       'ca', 'cp_0', 'cp_1', 'cp_2', 'cp_3', 'restecg_0', 'restecg_1',
       'restecg_2', 'slope_0', 'slope_1', 'slope_2', 'thal_0', 'thal_1',
       'thal_2', 'thal_3'],
      dtype='object')

In [165]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_ohe, y, test_size=0.2, random_state=0
)


In [166]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


In [167]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score


In [168]:
def build_classifier():
  classifier = Sequential()
  classifier.add(keras.Input(shape=(23,)))
  classifier.add(Dense(11, activation="relu"))
  classifier.add(Dropout(0.1))
  classifier.add(Dense(11, activation="relu"))
  classifier.add(Dropout(0.1))
  classifier.add(Dense(1, activation="sigmoid"))
  classifier.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
  return classifier


In [169]:
classifier = KerasClassifier(build_fn=build_classifier, batch_size=10, epochs=100)
accuracies = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10, n_jobs=-1)
mean = accuracies.mean()
variance = accuracies.std()
print(mean)
print(variance)

C:\Users\ishaa\AppData\Local\Temp\ipykernel_14736\3955631495.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn=build_classifier, batch_size=10, epochs=100)


0.8061666667461396
0.05066802848718733


In [170]:
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)


Epoch 1/100
25/25 [==============================] - 0s 4ms/step - loss: 0.9254 - accuracy: 0.4380
Epoch 2/100
25/25 [==============================] - 0s 4ms/step - loss: 0.8195 - accuracy: 0.4421
Epoch 3/100
25/25 [==============================] - 0s 4ms/step - loss: 0.7348 - accuracy: 0.4835
Epoch 4/100
25/25 [==============================] - 0s 4ms/step - loss: 0.6860 - accuracy: 0.5455
Epoch 5/100
25/25 [==============================] - 0s 4ms/step - loss: 0.6365 - accuracy: 0.7066
Epoch 6/100
25/25 [==============================] - 0s 4ms/step - loss: 0.6120 - accuracy: 0.7397
Epoch 7/100
25/25 [==============================] - 0s 4ms/step - loss: 0.5668 - accuracy: 0.7521
Epoch 8/100
25/25 [==============================] - 0s 4ms/step - loss: 0.5502 - accuracy: 0.7686
Epoch 9/100
25/25 [==============================] - 0s 5ms/step - loss: 0.5266 - accuracy: 0.7603
Epoch 10/100
25/25 [==============================] - 0s 5ms/step - loss: 0.4860 - accuracy: 0.8017
Epoch 11/

In [171]:
y_pred = y_pred > 0.5


In [172]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)


In [173]:
cm


array([[22,  5],
       [ 3, 31]], dtype=int64)

In [174]:
acc = accuracy_score(y_test, y_pred) * 100


In [175]:
acc


86.88524590163934